## Actividad 3: Entrenamiento de red neuronal convolucional (CNN)

Las CNN son arquitecturas de redes neuronales muy utilizadas para el tratamiento de imagenes, son arquitecturas muy potentes y de grandes prestaciones en el machine learning para resolver problemas de aprendizaje supervisado.

En el siguiente ejercicio entrenaremos un modelo que es capas de detectar si en una imagen hay un gato o no hay gato.

# Clonar repositorio donde se encuentra el dataset

In [ ]:
!git clone https://github.com/eyberthrojas/deeplearning-actividad-3.git

In [58]:
import pickle
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Conv2D, MaxPooling2D, Flatten

import matplotlib.pyplot as plt 

In [4]:
File = open('deeplearning-actividad-3/data.pckl', 'rb')
dat_X = pickle.load(File)
dat_Y = pickle.load(File)
File.close()
del(File)

In [ ]:
print(f'Dimenciones de la matriz de Imagenes = {dat_X.shape}')
print(f'Dimenciones de los Labels o Etiquetas = {dat_Y.shape}')

- dat_X es un tensor de 4 dimenciones, el cual tiene información de 259 imagenes de 64 pixeles por 64 pixeles y tres capas (RGB)
- dat_Y es un tensor de una dimencion de 259 posiciones y contiene las etiquetas (labels) de las imagenes contenidas en dat_X, (1: Es gato, 0: No es gato)

In [ ]:
# veamos las imagenes
index = 9   ### Se puede cambiar el indice (0-258)
plt.figure(figsize=(3,3))
plt.imshow(dat_X[index])
if dat_Y[index, 0]:
    print ("Es gato")
else:
    print ("NO es gato")

Division de datos de entrenamiento y test

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(dat_X, dat_Y, test_size=0.3, random_state=1, stratify=dat_Y) 
print('Train:{}, Test {}'.format(X_train.shape, X_test.shape))

In [ ]:
print(f'Numero de imagenes de gato para entrenamiento = {np.sum(Y_train)}')
print(f'Numero de imagenes de gato para test = {np.sum(Y_test)}')

Hyperparametros

In [ ]:
learning_rate = 0.0001
num_epochs = 20
hidden_layer_size = 256    # Número de neuronas en capa oculta
filters = [32, 128, 256]
orders = [5, 3, 3]
input_shape = X_train.shape[1:]
n_class = 1
print(f'Input Shape = {input_shape}')

## Costruccion del modelo

In [47]:
model = Sequential()
model.add(InputLayer(input_shape))
# Capas convolucionales, filter es el numero de imagenes de salida, order es el orden del kernel de la convolucion
# padding same hace que las imagenes de salida sean del mismo tamano de la de entrada 
for filter, order in zip(filters, orders):
  model.add(Conv2D(filter, order, padding='same', activation='relu'))
  model.add(MaxPooling2D((2, 2))) # Reduce a la mitad las imagenes de salida
model.add(Flatten())
model.add(Dense(hidden_layer_size, activation='relu'))
model.add(Dense(hidden_layer_size, activation='relu'))
model.add(Dense(n_class, activation='sigmoid'))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)


In [ ]:
model.summary()

## Entrenamiento

In [ ]:
history = model.fit(x=X_train, y=Y_train, batch_size=32, epochs=num_epochs, validation_data=(X_test, Y_test), verbose=True)

In [51]:
epochs_v = range(1,num_epochs+1)

In [ ]:
history.history.keys()

## Perdida vs epochs

In [ ]:
plt.plot(epochs_v, history.history['loss'], epochs_v, history.history['val_loss'])
#plt.legend(['train_loss', 'test_loss'])

## Accuracy vs epochs

In [ ]:
plt.plot(epochs_v, history.history['binary_accuracy'], epochs_v, history.history['val_binary_accuracy'])
#plt.legend(['train_accuracy', 'test_accuracy'])

## Predict con datos de test

In [ ]:
probs = model.predict(X_test)
preds = np.greater(probs, 0.5)

In [ ]:
print(classification_report(Y_test, preds, zero_division=0))